In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
import pandas as pd
import numpy as np
from sklearn import preprocessing,cross_decomposition,model_selection,metrics
import matplotlib.pyplot as plt
import seaborn as sns
# import xgboost as xgb
%matplotlib inline
import dateutil
from tqdm import tqdm
from sklearn import linear_model
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import tsa
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [76]:
confirmedGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',encoding='utf-8',na_values=None)

In [77]:
confirmedGlobal.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/5/22,1/6/22,1/7/22,1/8/22,1/9/22,1/10/22,1/11/22,1/12/22,1/13/22,1/14/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,158245,158275,158300,158309,158381,158394,158471,158511,158602,158639
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,212021,213257,214905,214905,219694,220487,222664,224569,226598,228777
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,220415,220825,221316,221742,222157,222639,223196,223806,224383,224979
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,25289,25289,26408,26408,26408,27983,28542,28899,28899,29888
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,86636,87625,88775,89251,89718,90316,91148,91907,92581,93302


In [78]:
bar_graph = confirmedGlobal[['Country/Region',confirmedGlobal.columns[-1]]].groupby('Country/Region').sum()

In [79]:
bar_graph = bar_graph.reset_index()
bar_graph.columns = ['Country/Region', 'values']

In [80]:
bar_graph = bar_graph.sort_values(by='values', ascending = False)
bar_graph['Country/Region'].values.tolist()

['US',
 'India',
 'Brazil',
 'United Kingdom',
 'France',
 'Russia',
 'Turkey',
 'Italy',
 'Spain',
 'Germany',
 'Argentina',
 'Iran',
 'Colombia',
 'Poland',
 'Indonesia',
 'Mexico',
 'Ukraine',
 'Netherlands',
 'South Africa',
 'Philippines',
 'Malaysia',
 'Canada',
 'Czechia',
 'Peru',
 'Belgium',
 'Thailand',
 'Iraq',
 'Vietnam',
 'Romania',
 'Chile',
 'Japan',
 'Portugal',
 'Israel',
 'Switzerland',
 'Greece',
 'Australia',
 'Bangladesh',
 'Sweden',
 'Serbia',
 'Slovakia',
 'Austria',
 'Hungary',
 'Pakistan',
 'Kazakhstan',
 'Denmark',
 'Jordan',
 'Ireland',
 'Morocco',
 'Cuba',
 'Georgia',
 'Nepal',
 'Bulgaria',
 'Lebanon',
 'Croatia',
 'United Arab Emirates',
 'Tunisia',
 'Bolivia',
 'Belarus',
 'Korea, South',
 'Guatemala',
 'Azerbaijan',
 'Costa Rica',
 'Saudi Arabia',
 'Ecuador',
 'Sri Lanka',
 'Lithuania',
 'Panama',
 'Burma',
 'Slovenia',
 'Norway',
 'Uruguay',
 'Dominican Republic',
 'Paraguay',
 'West Bank and Gaza',
 'Kuwait',
 'Ethiopia',
 'Venezuela',
 'Mongolia',
 'Eg

In [81]:
bar_graph['values'].values.tolist()

[64917963,
 36850962,
 22933289,
 15165209,
 13680775,
 10565484,
 10273170,
 8356514,
 8093036,
 7943959,
 6932972,
 6217320,
 5475904,
 4281482,
 4269740,
 4257776,
 3919151,
 3577487,
 3552043,
 3129512,
 2802263,
 2735428,
 2573945,
 2473709,
 2410731,
 2308615,
 2108206,
 1991484,
 1885771,
 1857764,
 1830381,
 1814567,
 1718982,
 1666441,
 1632641,
 1613249,
 1609042,
 1560363,
 1412588,
 1411568,
 1411421,
 1327014,
 1320120,
 1120948,
 1096434,
 1095087,
 1078181,
 1034399,
 992125,
 984802,
 845501,
 813840,
 808612,
 802745,
 799065,
 771177,
 725459,
 713002,
 687984,
 648947,
 624208,
 609882,
 604672,
 602942,
 594996,
 559980,
 558676,
 532851,
 519714,
 496312,
 490626,
 489631,
 475686,
 474234,
 456311,
 454556,
 452430,
 402462,
 397778,
 396452,
 386905,
 382440,
 371135,
 347084,
 316700,
 313185,
 310338,
 302872,
 298589,
 293695,
 291214,
 290030,
 262482,
 250009,
 241651,
 237678,
 228777,
 225637,
 224979,
 221416,
 217821,
 203297,
 200535,
 188206,
 164705,


In [82]:
country_names = bar_graph['Country/Region'].values.tolist()
cases = bar_graph['values'].values.tolist()

In [83]:
bar_graph

,Country/Region,values
182,US,64917963
79,India,36850962
23,Brazil,22933289
186,United Kingdom,15165209
62,France,13680775
...,...,...
112,Marshall Islands,7
149,Samoa,3
91,Kiribati,2
116,Micronesia,1


In [91]:
row = confirmedGlobal[confirmedGlobal['Country/Region'] == 'India']
# print(row)
data = []
data_csv = bar_graph
data_csv.reset_index()
data_csv.columns = ['date', 'cases']
data_csv = data_csv.drop(range(0, 196))
date = "20"
for i, col in enumerate(row):
    cases = list(row[col])[0]
    t = {}
    if i > 3 and cases != 0:
        new_cases = 0
        if len(data_csv) > 1 :
            new_cases = cases - (data[-1])['cases']

        datetime_obj = datetime.datetime.strptime(str(col),"%m/%d/%y")
        # print(datetime_obj)
        t['cases'] = cases
        t['date'] = datetime_obj
        data_csv.loc[0] = [datetime_obj, new_cases]
        data_csv.index = data_csv.index + 1
        # print(col, cases)
    data.append(t)
    # print(data)

    # data_csv.loc[-1] = [2, 3]  # adding a row
    data_csv.index = data_csv.index + 1  # shifting index
print(data_csv)
fig = go.Figure([go.Scatter(x=data_csv['date'], y=data_csv['cases'])])
fig.show()

           date   cases
1432 2020-01-30       0
1430 2020-01-31       0
1428 2020-02-01       0
1426 2020-02-02       1
1424 2020-02-03       1
...         ...     ...
10   2022-01-10  168063
8    2022-01-11       0
6    2022-01-12  442137
4    2022-01-13       0
2    2022-01-14  533035

[716 rows x 2 columns]


In [85]:
# Total Cases Wordlwide
confirmedGlobal[confirmedGlobal.columns[-1]].sum()

323567887

In [86]:
data = pd.read_json("https://cdn.jsdelivr.net/gh/highcharts/highcharts@v7.0.0/samples/data/world-population-density.json")


In [87]:
data.head(1)

,code3,name,value,code
0,ABW,Aruba,582.34,AW


In [88]:
data[data['name'] == 'Russia']

,code3,name,value,code


In [89]:
bar_graph[bar_graph['Country/Region'] == 'Greenland']

KeyError: 'Country/Region'

In [ ]:
x = bar_graph[bar_graph['Country/Region'] == 'Russia'] 
x['Country/Region'] = "ROC"
# bar_graph[bar_graph['Country/Region'] == 'Russia'] 
# confirmedGlobal.head(1)
t = confirmedGlobal

# t.loc[0, 'Country/Region'] = 'hi'
# t['Country/Region']['hi'] = t['Country/Region']['hi'].replace('hello')
# t.head(1)
bar_graph

In [ ]:
heat_map = []
c = 0
# for val in data:
#     if data 

rdf = data[data['name'] == "Russian Federation"]
for country in country_names:
    try :
        df = data[data['name'] == country]
        temp = {}
        temp['code3'] = list(df['code3'].values)[0]
        temp['code'] = list(df['code'].values)[0]
        temp['name'] = country
        temp['value'] = bar_graph[bar_graph['Country/Region'] == country]['values'].sum()
        # Or
        # tdf = bar_graph[bar_graph['Country/Region'] == country]
        # temp['value'] = list(tdf['values'].values)[0]
        heat_map.append(temp)
        c += 1
        # print(country)
    except:
        pass
# for country in country_names:
#     if data[data['name'] == country] is not None:
#         c += 1
print(c)



In [ ]:
heat_map